In [3058]:
import sys
import os
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.backends.cudnn as cudnn
import torch.optim as optim
import torch.optim.lr_scheduler as lr_scheduler
from torch.utils.data import Dataset, DataLoader

import torchvision
import torchvision.datasets
import torchvision.transforms as transforms

import numpy as np
import matplotlib.pyplot as plt

import time

from snntorch import spikegen
import matplotlib.pyplot as plt
import snntorch.spikeplot as splt
from IPython.display import HTML

from tqdm import tqdm


from modules.data_loader import *
from modules.network import *
from modules.neuron import *
from modules.synapse import *

In [3051]:
class SYNAPSE_CONV_gra_test(nn.Module):
    def __init__(self, in_channels, out_channels, kernel_size=3, stride=1, padding=1):
        super(SYNAPSE_CONV_gra_test, self).__init__()
        self.in_channels = in_channels
        self.out_channels = out_channels
        self.kernel_size = kernel_size
        self.stride = stride
        self.padding = padding
        self.weight = nn.Parameter(torch.randn(self.out_channels, self.in_channels, self.kernel_size, self.kernel_size))
        self.bias = nn.Parameter(torch.randn(self.out_channels))

    def forward(self, spike):
        return SYNAPSE_CONV_METHOD_gra_test.apply(spike, self.weight, self.bias, self.stride, self.padding)

class SYNAPSE_CONV_METHOD_gra_test(torch.autograd.Function):
    @staticmethod
    def forward(ctx, spike_one_time, weight, bias, stride=1, padding=1):
        ctx.save_for_backward(spike_one_time, weight, bias, torch.tensor([stride], requires_grad=False), torch.tensor([padding], requires_grad=False))
        return F.conv2d(spike_one_time, weight, bias=bias, stride=stride, padding=padding)

    @staticmethod
    def backward(ctx, grad_output_current):
        spike_one_time, weight, bias, stride, padding = ctx.saved_tensors
        stride=stride.item()
        padding=padding.item()
        
        ## 이거 클론해야되는지 모르겠음!!!!
        grad_output_current_clone = grad_output_current.clone()


        grad_input_spike = grad_weight = grad_bias = None

        if ctx.needs_input_grad[0]:
            grad_input_spike = F.conv_transpose2d(grad_output_current_clone, weight, stride=stride, padding=padding)
        if ctx.needs_input_grad[1]:
            grad_weight = torch.nn.grad.conv2d_weight(spike_one_time, weight.shape, grad_output_current_clone,
                                                    stride=stride, padding=padding)
        if bias is not None and ctx.needs_input_grad[2]:
            grad_bias = grad_output_current_clone.sum((0, -1, -2))

        # print('grad_input_spike_conv', grad_input_spike)
        # print('grad_weight_conv', grad_weight)
        # print('grad_bias_conv', grad_bias)
        # print('grad_input_spike_conv', ctx.needs_input_grad[0])
        # print('grad_weight_conv', ctx.needs_input_grad[2])
        # print('grad_bias_conv', ctx.needs_input_grad[3])

        return grad_input_spike, grad_weight, grad_bias, None, None
   

In [3052]:
batch = 3
in_channels = 2
out_channels = 4

image_size = 5

input_tensor = torch.randn(batch, in_channels, image_size, image_size, requires_grad=True)


# Define custom convolution layer
custom_conv = SYNAPSE_CONV_gra_test(in_channels=in_channels, out_channels=out_channels, kernel_size=3, stride=1, padding=1)
custom_conv_output = custom_conv(input_tensor)

# Define standard convolution layer with the same weights and biases as custom layer
standard_conv = nn.Conv2d(in_channels=in_channels, out_channels=out_channels, kernel_size=3, stride=1, padding=1)
standard_conv.weight = nn.Parameter(custom_conv.weight.data.clone())
standard_conv.bias = nn.Parameter(custom_conv.bias.data.clone())
standard_conv_output = standard_conv(input_tensor)

# Compare forward outputs
print("Forward outputs are equal: ", torch.allclose(custom_conv_output, standard_conv_output))

# Compute gradients
grad_output = torch.randn_like(custom_conv_output)
custom_conv_output.backward(grad_output, retain_graph=True)
standard_conv_output.backward(grad_output)

# Compare gradients w.r.t. input
print("Input gradients are equal: ", torch.allclose(input_tensor.grad, input_tensor.grad))

# Compare gradients w.r.t. weights
print("Weight gradients are equal: ", torch.allclose(custom_conv.weight.grad, standard_conv.weight.grad))

# Compare gradients w.r.t. biases
print("Bias gradients are equal: ", torch.allclose(custom_conv.bias.grad, standard_conv.bias.grad))

Forward outputs are equal:  True
Input gradients are equal:  True
Weight gradients are equal:  True
Bias gradients are equal:  True


In [3053]:
import torch
import torch.nn as nn
import torch.nn.functional as F
class CustomTwoLayerConvNet(nn.Module):
    def __init__(self, in_channels, hidden_channels, out_channels, kernel_size=3, stride=1, padding=1):
        super(CustomTwoLayerConvNet, self).__init__()
        self.layer1 = SYNAPSE_CONV_gra_test(in_channels, hidden_channels, kernel_size, stride, padding)
        self.layer2 = SYNAPSE_CONV_gra_test(hidden_channels, out_channels, kernel_size, stride, padding)
    
    def forward(self, x):
        x = self.layer1(x)
        x = F.relu(x)  # Adding a non-linearity for completeness
        x = self.layer2(x)
        return x

class StandardTwoLayerConvNet(nn.Module):
    def __init__(self, in_channels, hidden_channels, out_channels, kernel_size=3, stride=1, padding=1):
        super(StandardTwoLayerConvNet, self).__init__()
        self.layer1 = nn.Conv2d(in_channels, hidden_channels, kernel_size, stride, padding)
        self.layer2 = nn.Conv2d(hidden_channels, out_channels, kernel_size, stride, padding)
    
    def forward(self, x):
        x = self.layer1(x)
        x = F.relu(x)  # Adding a non-linearity for completeness
        x = self.layer2(x)
        return x

# Parameters
batch = 1
in_channels = 5
hidden_channels = 5
out_channels = 5
image_size = 5


# Define input tensor
input_tensor = torch.randn(batch, in_channels, image_size, image_size, requires_grad=True)

# Define custom two-layer convolution model
custom_model = CustomTwoLayerConvNet(in_channels, hidden_channels, out_channels)
custom_model_output = custom_model(input_tensor)

# Define standard two-layer convolution model with the same weights and biases as custom model
standard_model = StandardTwoLayerConvNet(in_channels, hidden_channels, out_channels)
standard_model.layer1.weight = nn.Parameter(custom_model.layer1.weight.data.clone())
standard_model.layer1.bias = nn.Parameter(custom_model.layer1.bias.data.clone())
standard_model.layer2.weight = nn.Parameter(custom_model.layer2.weight.data.clone())
standard_model.layer2.bias = nn.Parameter(custom_model.layer2.bias.data.clone())
standard_model_output = standard_model(input_tensor)

# Compare forward outputs
print("Forward outputs are equal: ", torch.allclose(custom_model_output, standard_model_output))

# Compute gradients
grad_output = torch.randn_like(custom_model_output)
custom_model_output.backward(grad_output, retain_graph=True)
standard_model_output.backward(grad_output)

input_tensor.grad= torch.round(input_tensor.grad * 1e5) / 1e5
custom_model.layer1.weight.grad= torch.round(custom_model.layer1.weight.grad * 1e5) / 1e5
standard_model.layer1.weight.grad= torch.round(standard_model.layer1.weight.grad * 1e5) / 1e5
custom_model.layer1.bias.grad= torch.round(custom_model.layer1.bias.grad * 1e5) / 1e5
standard_model.layer1.bias.grad= torch.round(standard_model.layer1.bias.grad * 1e5) / 1e5
custom_model.layer2.weight.grad= torch.round(custom_model.layer2.weight.grad * 1e5) / 1e5
standard_model.layer2.weight.grad= torch.round(standard_model.layer2.weight.grad * 1e5) / 1e5
custom_model.layer2.bias.grad= torch.round(custom_model.layer2.bias.grad * 1e5) / 1e5
standard_model.layer2.bias.grad= torch.round(standard_model.layer2.bias.grad * 1e5) / 1e5










# Compare gradients w.r.t. input
print("Input gradients are equal: ", torch.allclose(input_tensor.grad, input_tensor.grad))

# Compare gradients w.r.t. first layer weights
print("First layer weight gradients are equal: ", torch.allclose(custom_model.layer1.weight.grad, standard_model.layer1.weight.grad))

# Compare gradients w.r.t. first layer biases
print("First layer bias gradients are equal: ", torch.allclose(custom_model.layer1.bias.grad, standard_model.layer1.bias.grad))

# Compare gradients w.r.t. second layer weights
print("Second layer weight gradients are equal: ", torch.allclose(custom_model.layer2.weight.grad, standard_model.layer2.weight.grad))

# Compare gradients w.r.t. second layer biases
print("Second layer bias gradients are equal: ", torch.allclose(custom_model.layer2.bias.grad, standard_model.layer2.bias.grad))




# print('custom_model.layer1.weight.grad', custom_model.layer1.weight.grad.size())
# print('standard_model.layer1.weight.grad', standard_model.layer1.weight.grad.size())
# print('custom_model.layer1.weight.grad', custom_model.layer1.weight.grad)
# print('standard_model.layer1.weight.grad', standard_model.layer1.weight.grad)


Forward outputs are equal:  True
Input gradients are equal:  True
First layer weight gradients are equal:  True
First layer bias gradients are equal:  True
Second layer weight gradients are equal:  True
Second layer bias gradients are equal:  True


In [3054]:
print("First layer weight gradients are equal: ", torch.allclose(custom_model.layer1.weight.grad, standard_model.layer1.weight.grad))

First layer weight gradients are equal:  True


In [3055]:
custom_model.layer1.weight.grad= torch.round(custom_model.layer1.weight.grad * 1e5) / 1e5
custom_model.layer1.weight.grad

tensor([[[[ -0.6744, -21.4038,   0.1721],
          [ -5.7564,  -8.3224, -28.2213],
          [-17.3416,  -2.1030,  -1.7222]],

         [[-15.6547,  -6.5354,  -4.7486],
          [ -0.1185,  -7.5696,  -1.3348],
          [  7.8487,  33.8435,   4.6007]],

         [[ 15.5353, -25.6578,   0.7505],
          [ 44.0459,   9.1942,  27.1311],
          [ -4.8918,  30.1869,  -6.4636]],

         [[ -4.4907, -34.3023,  11.5691],
          [-26.3487, -20.6629,  15.6554],
          [-10.1458,  -4.0989,  15.2669]],

         [[ 13.6383,  -7.3613,  11.5022],
          [  9.8657,   0.8761,  17.5641],
          [-16.2694,   8.6354,   3.4830]]],


        [[[ 13.0467,  -8.8762, -18.4378],
          [  5.6352,  33.0969, -13.9500],
          [-28.1104,   6.1755,  22.3356]],

         [[ -8.7892, -18.6530,  -9.2455],
          [ 35.9025,  15.3442,   7.3019],
          [  7.5832,  28.8663,  10.8979]],

         [[  8.0235, -16.2218, -16.0273],
          [ 39.4245,  18.9508, -31.5385],
          [-31.039

In [3056]:
standard_model.layer1.weight.grad = torch.round(standard_model.layer1.weight.grad * 1e5) / 1e5
standard_model.layer1.weight.grad

tensor([[[[ -0.6744, -21.4038,   0.1721],
          [ -5.7564,  -8.3224, -28.2213],
          [-17.3416,  -2.1030,  -1.7222]],

         [[-15.6547,  -6.5354,  -4.7486],
          [ -0.1185,  -7.5696,  -1.3348],
          [  7.8487,  33.8435,   4.6007]],

         [[ 15.5353, -25.6578,   0.7505],
          [ 44.0459,   9.1942,  27.1311],
          [ -4.8918,  30.1869,  -6.4636]],

         [[ -4.4907, -34.3023,  11.5691],
          [-26.3487, -20.6629,  15.6554],
          [-10.1458,  -4.0989,  15.2669]],

         [[ 13.6383,  -7.3613,  11.5022],
          [  9.8657,   0.8761,  17.5641],
          [-16.2694,   8.6354,   3.4830]]],


        [[[ 13.0467,  -8.8762, -18.4378],
          [  5.6352,  33.0969, -13.9500],
          [-28.1104,   6.1755,  22.3356]],

         [[ -8.7892, -18.6530,  -9.2455],
          [ 35.9025,  15.3442,   7.3019],
          [  7.5832,  28.8663,  10.8979]],

         [[  8.0235, -16.2218, -16.0273],
          [ 39.4245,  18.9508, -31.5385],
          [-31.039

In [3059]:
custom_model.layer1.weight.grad[0][0][0][0] 

tensor(-0.6744)

In [3060]:
standard_model.layer1.weight.grad[0][0][0][0]

tensor(-0.6744)

In [3057]:
custom_model.layer1.weight.grad == standard_model.layer1.weight.grad

tensor([[[[ True,  True,  True],
          [ True,  True,  True],
          [ True,  True,  True]],

         [[ True,  True,  True],
          [ True,  True,  True],
          [ True,  True,  True]],

         [[ True,  True,  True],
          [ True,  True,  True],
          [ True,  True,  True]],

         [[ True,  True,  True],
          [ True,  True,  True],
          [ True,  True,  True]],

         [[ True,  True,  True],
          [ True,  True,  True],
          [ True,  True,  True]]],


        [[[ True,  True,  True],
          [ True,  True,  True],
          [ True,  True,  True]],

         [[ True,  True,  True],
          [ True,  True,  True],
          [ True,  True,  True]],

         [[ True,  True,  True],
          [ True,  True,  True],
          [ True,  True,  True]],

         [[ True,  True,  True],
          [ True,  True,  True],
          [ True,  True, False]],

         [[ True,  True,  True],
          [ True,  True,  True],
          [ True,  True